In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'type': ['A', 'B', 'A'],
    'value': [10.5, 20.3, 30.7]
})
df1['type'] = df1['type'].astype('category')

df2 = pd.DataFrame({
    'id': ['1', '2', '3'],             
    'type': ['A', 'B', 'E'],           
    'value': ['10.5', '20.3', '30.7']
})


dtypes_out_ = df1.dtypes.to_dict()
df_2_aligned = df2.astype(dtypes_out_)

print(df1.dtypes)
print(df_2_aligned.dtypes)
print({cat: code for code, cat in enumerate(df1['type'].cat.categories)})
print({cat: code for code, cat in enumerate(df_2_aligned['type'].cat.categories)})

id          int64
type     category
value     float64
dtype: object
id          int64
type     category
value     float64
dtype: object
{'A': 0, 'B': 1}
{'A': 0, 'B': 1}


In [11]:
df_2_aligned

,id,type,value
0,1,A,10.5
1,2,B,20.3
2,3,NaN,30.7


In [12]:
df2 = pd.DataFrame({
    'id': ['1', '2', '3'],             
    'type': ['A', 'A', 'A'],           
    'value': ['10.5', '20.3', '30.7']
})


dtypes_out_ = df1.dtypes.to_dict()
df_2_aligned = df2.astype(dtypes_out_)
df2

,id,type,value
0,1,A,10.5
1,2,A,20.3
2,3,A,30.7


In [13]:
print({cat: code for code, cat in enumerate(df_2_aligned['type'].cat.categories)})

{'A': 0, 'B': 1}


In [15]:
np.array([True, False, 1.0])

array([1., 0., 1.])

In [4]:
df1 = pd.DataFrame({
    'id': [1, 2, 3] * 10000,
    'type': ['A', 'B', 'A'] * 10000,
    'value': [10.5, 20.3, 30.7] * 10000
})
df1['type'] = df1['type'].astype('category')

In [5]:
%%timeit

df1.dtypes.to_dict()

35.5 μs ± 760 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
n_rows = 1000000
n_cols = 50

df1 = pd.DataFrame(np.random.rand(n_rows, n_cols), columns=[f'col_{i}' for i in range(n_cols)])

# Cast 10 columns to 'category'
for i in range(10):
    df1[f'col_{i}'] = df1[f'col_{i}'].astype('category')


In [7]:
%%timeit

df1.dtypes.to_dict()

62.4 μs ± 811 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
exog_dtypes_out_ = df1.dtypes.to_dict()

In [18]:
%%timeit

any(
not pd.api.types.is_numeric_dtype(dtype) or pd.api.types.is_bool_dtype(dtype) 
for dtype in set(exog_dtypes_out_)
)

31.7 μs ± 600 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.utils import transform_numpy
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import y as y_categorical
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import exog as exog_categorical
from skforecast.recursive.tests.tests_forecaster_recursive.fixtures_forecaster_recursive import data  # to test results when using differentiation

In [20]:
df_exog = pd.DataFrame(
        {'exog_1': exog_categorical,
         'exog_2': ['a', 'b', 'c', 'd', 'e'] * 10,
         'exog_3': pd.Categorical(['F', 'G', 'H', 'I', 'J'] * 10)}
    )
    
exog_predict = df_exog.copy()
exog_predict.index = pd.RangeIndex(start=50, stop=100)

pipeline_categorical = make_pipeline(
                        OrdinalEncoder(
                            dtype=int,
                            handle_unknown="use_encoded_value",
                            unknown_value=-1,
                            encoded_missing_value=-1
                        ),
                        FunctionTransformer(
                            func=lambda x: x.astype('category'),
                            feature_names_out= 'one-to-one'
                        )
                    )

transformer_exog = make_column_transformer(
                        (
                            pipeline_categorical,
                            make_column_selector(dtype_exclude=np.number)
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                    ).set_output(transform="pandas")

forecaster = ForecasterRecursive(
                    regressor        = LGBMRegressor(verbose=-1, random_state=123),
                    lags             = 5,
                    transformer_y    = None,
                    transformer_exog = transformer_exog
                )
forecaster.fit(y=y_categorical, exog=df_exog)
results = forecaster.create_predict_X(steps=10, exog=exog_predict)

In [21]:
results

,lag_1,lag_2,lag_3,lag_4,lag_5,exog_2,exog_3,exog_1
50,0.612895,0.519485,0.985560,0.483034,0.250455,0,0,0.120629
51,0.585703,0.612895,0.519485,0.985560,0.483034,1,1,0.826341
52,0.389450,0.585703,0.612895,0.519485,0.985560,2,2,0.603060
53,0.450534,0.389450,0.585703,0.612895,0.519485,3,3,0.545068
54,0.496866,0.450534,0.389450,0.585703,0.612895,4,4,0.342764
55,0.458875,0.496866,0.450534,0.389450,0.585703,0,0,0.304121
56,0.514811,0.458875,0.496866,0.450534,0.389450,1,1,0.417022
57,0.585703,0.514811,0.458875,0.496866,0.450534,2,2,0.681301
58,0.389450,0.585703,0.514811,0.458875,0.496866,3,3,0.875457
59,0.450534,0.389450,0.585703,0.514811,0.458875,4,4,0.510422


In [23]:
results.to_numpy()

array([[0.61289453, 0.51948512, 0.98555979, 0.48303426, 0.25045537,
        0.        , 0.        , 0.12062867],
       [0.5857033 , 0.61289453, 0.51948512, 0.98555979, 0.48303426,
        1.        , 1.        , 0.8263408 ],
       [0.3894503 , 0.5857033 , 0.61289453, 0.51948512, 0.98555979,
        2.        , 2.        , 0.60306013],
       [0.45053399, 0.3894503 , 0.5857033 , 0.61289453, 0.51948512,
        3.        , 3.        , 0.54506801],
       [0.49686551, 0.45053399, 0.3894503 , 0.5857033 , 0.61289453,
        4.        , 4.        , 0.34276383],
       [0.45887492, 0.49686551, 0.45053399, 0.3894503 , 0.5857033 ,
        0.        , 0.        , 0.30412079],
       [0.51481068, 0.45887492, 0.49686551, 0.45053399, 0.3894503 ,
        1.        , 1.        , 0.41702221],
       [0.5857033 , 0.51481068, 0.45887492, 0.49686551, 0.45053399,
        2.        , 2.        , 0.68130077],
       [0.3894503 , 0.5857033 , 0.51481068, 0.45887492, 0.49686551,
        3.        , 3.      

In [22]:
results.dtypes

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_2    category
exog_3    category
exog_1     float64
dtype: object

In [7]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'type': ['A', 'B', 'A'],
    'value': [10.5, 20.3, 30.7]
})
df1['type'] = df1['type'].astype('category')

df2 = pd.DataFrame({
    'id': ['1', '2', '3'],             
    'type': ['A', 'B', 'C'],           
    'value': ['10.5', '20.3', '30.7']
})
df2['type'] = df2['type'].astype('category')

print(df1.dtypes)
print(df2.dtypes)
print({cat: code for code, cat in enumerate(df1['type'].cat.categories)})
print({cat: code for code, cat in enumerate(df2['type'].cat.categories)})

id          int64
type     category
value     float64
dtype: object
id         object
type     category
value      object
dtype: object
{'A': 0, 'B': 1}
{'A': 0, 'B': 1, 'C': 2}


In [8]:
pd.concat([df1, df2]).dtypes

id       object
type     object
value    object
dtype: object

In [84]:
exog_dict = {
    'serie_1': pd.DataFrame({
                'id': ['1', '2', '3'],             
                'type': ['A', 'B', 'A'],           
                'value': ['10.5', '20.3', '30.7']
            }).astype({'type': 'category'}),
    'serie_2': pd.DataFrame({
                'id': ['1', '2', '3'],             
                'type': ['A', 'B', 'C'],        
            })
}
exog_dict['serie_1'].dtypes

id         object
type     category
value      object
dtype: object

In [85]:
exog_dtypes_buffer = [df.dtypes.astype(str) for df in exog_dict.values() if df is not None]
exog_dtypes_buffer

[id         object
 type     category
 value      object
 dtype: object,
 id      object
 type    object
 dtype: object]

In [86]:
exog_dtypes_buffer[0].iat[0]

'object'

In [87]:
exog_dict['serie_1'].dtypes['type'] == exog_dict['serie_2'].dtypes['type']

False

In [88]:
exog_dict['serie_2'].dtypes['type']

dtype('O')

In [89]:
exog_dtypes_buffer = pd.concat(exog_dtypes_buffer, axis=1)
exog_dtypes_buffer

,0,1
id,object,object
type,category,object
value,object,NaN


In [83]:
exog_dtypes_buffer.apply(lambda x: np.unique(x), axis=1)

id                 [object]
type     [category, object]
value              [object]
dtype: object

In [94]:
# Si cada celda de exog_dtypes_buffer es una lista/array
result = exog_dtypes_buffer.map(np.unique).apply(lambda x: x.tolist())
result

,0,1
id,[object],[object]
type,[category],[object]
value,[object],[nan]


In [54]:
exog_dtypes_nunique = pd.concat(exog_dtypes_buffer, axis=1).nunique(axis=1).eq(1)
exog_dtypes_nunique

id       True
type     True
value    True
dtype: bool

In [38]:
exog_dtypes_nunique[exog_dtypes_nunique != 1].index.to_list()

['type']

In [95]:
exog_dict = {
    'serie_1': pd.DataFrame({
                'id': ['1', '2', '3'],             
                'type': ['A', 'B', 'A'],           
                'value': ['10.5', '20.3', '30.7']
            }).astype({'type': 'category'}),
    'serie_2': pd.DataFrame({
                'id': ['1', '2', '3'],             
                'type': ['A', 'B', 'C'],        
            })
}
exog_dict['serie_1'].dtypes

id         object
type     category
value      object
dtype: object

In [106]:
serie_1 = pd.DataFrame({
            'id': ['1', '2', '3'] * 1000,             
            'type': ['A', 'B', 'A'] * 1000,           
            'value': ['10.5', '20.3', '30.7'] * 1000
        }).astype({'type': 'category'})
serie_2 = pd.DataFrame({
            'id': ['1', '2', '3'] * 1000,             
            'type': ['A', 'B', 'C'] * 1000,        
        })

exog_dict = {
    f'serie_{i}': serie_1.copy() if i % 2 == 0 else serie_2.copy()
    for i in range(1, 1000)
}

In [116]:
serie_1['type'].cat.categories

Index(['A', 'B'], dtype='object')

In [111]:
%%timeit

exog_dtypes_buffer = [df.dtypes for df in exog_dict.values() if df is not None]
exog_dtypes_buffer = pd.concat(exog_dtypes_buffer, axis=1)
exog_dtypes_nunique = exog_dtypes_buffer.nunique(axis=1).eq(1)
if not exog_dtypes_nunique.all():
    non_unique_dtypes_exogs = exog_dtypes_nunique[exog_dtypes_nunique != 1].index.to_list()

94.7 ms ± 6.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [115]:
exog_dtypes_buffer = pd.DataFrame(
    {k: df.dtypes for k, df in exog_dict.items() if df is not None}
)
exog_dtypes_nunique = exog_dtypes_buffer.nunique(axis=1)
if not (exog_dtypes_nunique == 1).all():
    non_unique_dtypes_exogs = exog_dtypes_nunique[exog_dtypes_nunique != 1].index.to_list()
    raise TypeError(
        f"Exog/s: {non_unique_dtypes_exogs} have different dtypes in different series."
    )

TypeError: Exog/s: ['type'] have different dtypes in different series.

In [113]:
%%timeit

exog_dtypes_buffer = pd.DataFrame(
    {k: df.dtypes for k, df in exog_dict.items() if df is not None}
)
exog_dtypes_nunique = exog_dtypes_buffer.nunique(axis=1).eq(1)
if not exog_dtypes_nunique.all():
    non_unique_dtypes_exogs = exog_dtypes_nunique[exog_dtypes_nunique != 1].index.to_list()

93.4 ms ± 4.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
pd.DataFrame(
        data = np.array([[4., 3., 2., 1., 0.],
                            [5., 4., 3., 2., 1.],
                            [6., 5., 4., 3., 2.],
                            [7., 6., 5., 4., 3.],
                            [8., 7., 6., 5., 4.],
                            [4., 3., 2., 1., 0.],
                            [5., 4., 3., 2., 1.],
                            [6., 5., 4., 3., 2.],
                            [7., 6., 5., 4., 3.],
                            [8., 7., 6., 5., 4.]]),
        index   = pd.Index([5, 6, 7, 8, 9, 5, 6, 7, 8, 9]),
        columns = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']
    ).assign(
        l1   = [1.] * 5 + [0.] * 5, 
        l2   = [0.] * 5 + [1.] * 5,
        exog = pd.Categorical([5, 6, 7, 8, 9] * 2, categories=range(10))
    ).astype({'l1': int, 'l2': int}).dtypes

lag_1     float64
lag_2     float64
lag_3     float64
lag_4     float64
lag_5     float64
l1          int64
l2          int64
exog     category
dtype: object

In [10]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10, dtype=float)), 
                           'l2': pd.Series(np.arange(10, dtype=float))})
exog = pd.Series(range(10), name='exog', dtype='category')

forecaster = ForecasterRecursiveMultiSeries(LinearRegression(), lags=5,
                                            encoding='onehot',
                                            transformer_series=None)
results = forecaster._create_train_X_y(series=series, exog=exog,
                                        store_last_window=False)   

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [12]:
results[0].dtypes

lag_1     float64
lag_2     float64
lag_3     float64
lag_4     float64
lag_5     float64
l1          int64
l2          int64
exog     category
dtype: object

In [17]:
X_train = pd.DataFrame(
    data = np.array([[4., 3., 2., 1., 0.],
                        [5., 4., 3., 2., 1.],
                        [6., 5., 4., 3., 2.],
                        [7., 6., 5., 4., 3.],
                        [8., 7., 6., 5., 4.],
                        [4., 3., 2., 1., 0.],
                        [5., 4., 3., 2., 1.],
                        [6., 5., 4., 3., 2.],
                        [7., 6., 5., 4., 3.],
                        [8., 7., 6., 5., 4.]]),
    index   = pd.Index([5, 6, 7, 8, 9, 5, 6, 7, 8, 9]),
    columns = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']
)
X_train.dtypes

lag_1    float64
lag_2    float64
lag_3    float64
lag_4    float64
lag_5    float64
dtype: object

In [19]:
X_train.astype({'lag_1': int}).dtypes

lag_1      int64
lag_2    float64
lag_3    float64
lag_4    float64
lag_5    float64
dtype: object

In [122]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries

from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_predict
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import expected_df_to_long_format

In [123]:
df_exog = pd.DataFrame(
        {'exog_1': exog['exog_1'],
         'exog_2': ['a', 'b'] * 25,
         'exog_3': pd.Categorical(['F', 'G', 'H', 'I', 'J'] * 10)}
    )
    
exog_predict = df_exog.copy()
exog_predict.index = pd.RangeIndex(start=50, stop=100)

pipeline_categorical = make_pipeline(
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            FunctionTransformer(
                                func=lambda x: x.astype('category'),
                                feature_names_out= 'one-to-one'
                            )
                        )

transformer_exog = make_column_transformer(
                        (
                            pipeline_categorical,
                            make_column_selector(dtype_exclude=np.number)
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                    ).set_output(transform="pandas")

forecaster = ForecasterRecursiveMultiSeries(
                    regressor          = LGBMRegressor(verbose=-1, random_state=123),
                    lags               = 5,
                    transformer_series = None,
                    transformer_exog   = transformer_exog
                )
forecaster.fit(series=series, exog=df_exog)
results = forecaster.create_predict_X(steps=10, exog=exog_predict)
results

,level,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast,exog_2,exog_3,exog_1
50,1,0.612895,0.519485,0.985560,0.483034,0.250455,0.0,0,0,0.513128
50,2,0.343456,0.240856,0.398876,0.151127,0.691702,1.0,0,0,0.513128
51,1,0.619929,0.612895,0.519485,0.985560,0.483034,0.0,1,1,0.666625
51,2,0.710577,0.343456,0.240856,0.398876,0.151127,1.0,1,1,0.666625
52,1,0.428036,0.619929,0.612895,0.519485,0.985560,0.0,0,2,0.105908
52,2,0.482054,0.710577,0.343456,0.240856,0.398876,1.0,0,2,0.105908
53,1,0.494009,0.428036,0.619929,0.612895,0.519485,0.0,1,3,0.130895
53,2,0.656553,0.482054,0.710577,0.343456,0.240856,1.0,1,3,0.130895
54,1,0.726535,0.494009,0.428036,0.619929,0.612895,0.0,0,4,0.321981
54,2,0.582494,0.656553,0.482054,0.710577,0.343456,1.0,0,4,0.321981


In [128]:
results[results['level'] == '2'][['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 
                            '_level_skforecast', 'exog_2', 'exog_3', 'exog_1']].to_numpy()

array([[0.34345601, 0.2408559 , 0.39887629, 0.15112745, 0.6917018 ,
        1.        , 0.        , 0.        , 0.51312815],
       [0.71057667, 0.34345601, 0.2408559 , 0.39887629, 0.15112745,
        1.        , 1.        , 1.        , 0.66662455],
       [0.48205396, 0.71057667, 0.34345601, 0.2408559 , 0.39887629,
        1.        , 0.        , 2.        , 0.10590849],
       [0.65655329, 0.48205396, 0.71057667, 0.34345601, 0.2408559 ,
        1.        , 1.        , 3.        , 0.13089495],
       [0.58249412, 0.65655329, 0.48205396, 0.71057667, 0.34345601,
        1.        , 0.        , 4.        , 0.32198061],
       [0.59331406, 0.58249412, 0.65655329, 0.48205396, 0.71057667,
        1.        , 1.        , 0.        , 0.66156434],
       [0.40887602, 0.59331406, 0.58249412, 0.65655329, 0.48205396,
        1.        , 0.        , 1.        , 0.84650623],
       [0.51731538, 0.40887602, 0.59331406, 0.58249412, 0.65655329,
        1.        , 1.        , 2.        , 0.55325734],


In [126]:
results[results['level'] == '1']

,level,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast,exog_2,exog_3,exog_1
50,1,0.612895,0.519485,0.985560,0.483034,0.250455,0.0,0,0,0.513128
51,1,0.619929,0.612895,0.519485,0.985560,0.483034,0.0,1,1,0.666625
52,1,0.428036,0.619929,0.612895,0.519485,0.985560,0.0,0,2,0.105908
53,1,0.494009,0.428036,0.619929,0.612895,0.519485,0.0,1,3,0.130895
54,1,0.726535,0.494009,0.428036,0.619929,0.612895,0.0,0,4,0.321981
55,1,0.546331,0.726535,0.494009,0.428036,0.619929,0.0,1,0,0.661564
56,1,0.387058,0.546331,0.726535,0.494009,0.428036,0.0,0,1,0.846506
57,1,0.463457,0.387058,0.546331,0.726535,0.494009,0.0,1,2,0.553257
58,1,0.680307,0.463457,0.387058,0.546331,0.726535,0.0,0,3,0.854452
59,1,0.421465,0.680307,0.463457,0.387058,0.546331,0.0,1,4,0.384838
